<a href="https://colab.research.google.com/github/edodoe15/Magistarski_Edin_Kohnic/blob/main/LSTM_%26_linear_regresija.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#LSTM
#biblioteke

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import plotly.graph_objects as go
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, Bidirectional, BatchNormalization
from kerastuner.tuners import RandomSearch


In [ ]:
#čišćenje i sezonske funkcije

def get_season(month): return (month % 12) // 3

def month_sin_cos(month):
    sin_m = np.sin(2 * np.pi * month / 12)
    cos_m = np.cos(2 * np.pi * month / 12)
    return sin_m, cos_m

def remove_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    return data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]


In [ ]:
#preprocesiranje i podjela podataka

def preprocess_data(filepath, look_back=120):
    df = pd.read_csv(filepath, usecols=['monthly_avg_temp', 'Year', 'Month'])
    df = remove_outliers_iqr(df)

    scaler = MinMaxScaler()
    df['scaled_temp'] = scaler.fit_transform(df[['monthly_avg_temp']])

    df['season'] = df['Month'].apply(get_season)
    df['sin_month'], df['cos_month'] = zip(*df['Month'].map(month_sin_cos))

    features, labels, meta = [], [], []

    for i in range(len(df) - look_back):
        temp_seq = df['scaled_temp'].values[i:i+look_back]
        target = df['scaled_temp'].values[i+look_back]
        row = df.iloc[i+look_back]
        features.append(np.concatenate([temp_seq, [row['season'], row['sin_month'], row['cos_month']]]))
        labels.append(target)
        meta.append([row['Year'], row['Month']])

    return np.array(features), np.array(labels), np.array(meta), scaler

def split_dataset(X, y, meta, train_ratio=0.7, val_ratio=0.15):
    total = len(X)
    train_end = int(train_ratio * total)
    val_end = train_end + int(val_ratio * total)

    return (
        X[:train_end], y[:train_end],
        X[train_end:val_end], y[train_end:val_end],
        X[val_end:], y[val_end:],
        meta[val_end:]
    )


In [ ]:
#inicijalizacija podataka

look_back = 120
X, y, meta, scaler = preprocess_data("data_bih_temp.csv", look_back)
trainX, trainY, valX, valY, testX, testY, testMeta = split_dataset(X, y, meta)

trainX = trainX.reshape(trainX.shape[0], 1, trainX.shape[1])
valX = valX.reshape(valX.shape[0], 1, valX.shape[1])
testX = testX.reshape(testX.shape[0], 1, testX.shape[1])


In [ ]:
#keras tuner definicija modela

def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', 32, 256, step=32), input_shape=(1, trainX.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [ ]:
tuner = RandomSearch(build_model, objective='val_loss', max_trials=5, executions_per_trial=1)
tuner.search(trainX, trainY, validation_data=(valX, valY), epochs=10)


Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



In [ ]:
trainPred = scaler.inverse_transform(best_model.predict(trainX))
testPred = scaler.inverse_transform(best_model.predict(testX))
trainY_actual = scaler.inverse_transform(trainY.reshape(-1, 1))
testY_actual = scaler.inverse_transform(testY.reshape(-1, 1))

print("Train RMSE:", np.sqrt(mean_squared_error(trainY_actual, trainPred)))
print("Test RMSE:", np.sqrt(mean_squared_error(testY_actual, testPred)))
print("Train MAPE:", mean_absolute_percentage_error(trainY_actual, trainPred))
print("Test MAPE:", mean_absolute_percentage_error(testY_actual, testPred))


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Train RMSE: 1.8861807385925173
Test RMSE: 1.7001013393372633
Train MAPE: 1.062865600007743
Test MAPE: 0.4150398683640997


In [ ]:
def plot_predictions(real, predicted, meta):
    fig = go.Figure()
    time_labels = meta[:, 0].astype(str) + '-' + meta[:, 1].astype(str)
    fig.add_trace(go.Scatter(x=time_labels, y=real.flatten(), name='Stvarna'))
    fig.add_trace(go.Scatter(x=time_labels, y=predicted.flatten(), name='Predikcija'))
    fig.update_layout(title='Stvarne vs. Predikovane temperature (LSTM)', xaxis_title='Vrijeme', yaxis_title='Temp (°C)')
    fig.show()

plot_predictions(testY_actual, testPred, testMeta)


In [ ]:
best_model.save("bih_temp_best_model.keras")

with open("predikcije_test.txt", "w") as f:
    f.write("Godina, Mjesec, Stvarna temperatura, Predikcija\n")
    for i in range(len(testY_actual)):
        f.write(f"{testMeta[i][0]}, {testMeta[i][1]}, {testY_actual[i][0]:.2f}, {testPred[i][0]:.2f}\n")


In [ ]:
future_years = np.arange(2026, 2046)
future_months = np.tile(np.arange(1, 13), len(future_years))
future_years_full = np.repeat(future_years, 12)
future_preds = []

last_scaled_temps = X[-1, :look_back]
last_input = np.concatenate([last_scaled_temps, [0,0,0]])

for year, month in zip(future_years_full, future_months):
    season = get_season(month)
    sin_m, cos_m = month_sin_cos(month)


    input_full = np.copy(last_input)
    input_full[-3:] = [season, sin_m, cos_m]

    input_full = input_full.reshape(1, 1, -1)

    pred_scaled = best_model.predict(input_full)
    pred_temp = scaler.inverse_transform(pred_scaled.reshape(-1, 1))[0][0]

    future_preds.append([year, month, pred_temp])

    norm_pred = scaler.transform([[pred_temp]])[0][0]

    last_input[:-4] = last_input[1:-3]
    last_input[-4] = norm_pred

with open("predikcije_2026_2045.txt", "w") as f:
    f.write("Godina, Mjesec, Predikcija temperature\n")
    for pred in future_preds:
        f.write(f"{pred[0]}, {pred[1]}, {pred[2]:.2f}\n")

print("Predikcije za 2026-2045 sačuvane u 'predikcije_2026_2045.txt'.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predikcije za 2026-2045 sačuvane u 'predikcije_2026_2045.txt'.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



In [ ]:
#Linearna regresija

import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

def get_season(month):
    if month in [12, 1, 2]: return 0
    elif month in [3, 4, 5]: return 1
    elif month in [6, 7, 8]: return 2
    else: return 3

def month_sin_cos(month):
    sin_m = np.sin(2 * np.pi * month / 12)
    cos_m = np.cos(2 * np.pi * month / 12)
    return sin_m, cos_m

def create_dataset(dataset, months, years, look_back=120):
    dataX, dataY, monthX, yearX, seasonX, sinMonthX, cosMonthX = [], [], [], [], [], [], []
    for i in range(len(dataset) - look_back - 1):
        dataX.append(dataset[i:(i + look_back), 0])
        dataY.append(dataset[i + look_back, 0])
        m = months[i + look_back]
        monthX.append(m)
        yearX.append(years[i + look_back])
        seasonX.append(get_season(m))
        sin_m, cos_m = month_sin_cos(m)
        sinMonthX.append(sin_m)
        cosMonthX.append(cos_m)
    return np.array(dataX), np.array(dataY), np.array(monthX), np.array(yearX), np.array(seasonX), np.array(sinMonthX), np.array(cosMonthX)

df = pd.read_csv("data_bih_temp.csv", usecols=["monthly_avg_temp", "Year", "Month"])
dataset = df["monthly_avg_temp"].values.astype("float32").reshape(-1, 1)
months = df["Month"].values
years = df["Year"].values

scaler = MinMaxScaler()
dataset_scaled = scaler.fit_transform(dataset)

look_back = 120
train_data = dataset_scaled[years < 2000]
train_months = months[years < 2000]
train_years = years[years < 2000]

test_data = dataset_scaled[years >= 2000]
test_months = months[years >= 2000]
test_years = years[years >= 2000]

trainX, trainY, trainMonthX, trainYearX, trainSeasonX, trainSinX, trainCosX = create_dataset(train_data, train_months, train_years, look_back)
testX, testY, testMonthX, testYearX, testSeasonX, testSinX, testCosX = create_dataset(test_data, test_months, test_years, look_back)

trainX_full = np.column_stack((trainX, trainSeasonX, trainSinX, trainCosX))
testX_full = np.column_stack((testX, testSeasonX, testSinX, testCosX))

model = Ridge(alpha=100)
model.fit(trainX_full, trainY)

train_pred = model.predict(trainX_full)
test_pred = model.predict(testX_full)

trainY_inv = scaler.inverse_transform(trainY.reshape(-1, 1))
testY_inv = scaler.inverse_transform(testY.reshape(-1, 1))
train_pred_inv = scaler.inverse_transform(train_pred.reshape(-1, 1))
test_pred_inv = scaler.inverse_transform(test_pred.reshape(-1, 1))

train_rmse = np.sqrt(mean_squared_error(trainY_inv, train_pred_inv))
test_rmse = np.sqrt(mean_squared_error(testY_inv, test_pred_inv))
train_mape = mean_absolute_percentage_error(trainY_inv, train_pred_inv)
test_mape = mean_absolute_percentage_error(testY_inv, test_pred_inv)

print(f"Train Score: {train_rmse:.2f} RMSE")
print(f"Test Score: {test_rmse:.2f} RMSE")
print(f"Train MAPE: {train_mape:.2f}%")
print(f"Test MAPE: {test_mape:.2f}%")

with open("linear_test_predikcije.txt", "w") as f:
    f.write("Godina, Mjesec, Stvarna temperatura, Predikcija\n")
    for i in range(len(testY)):
        f.write(f"{testYearX[i]}, {testMonthX[i]}, {testY_inv[i][0]:.2f}, {test_pred_inv[i][0]:.2f}\n")

print("Testne predikcije sačuvane u 'linear_test_predikcije.txt'.")

future_years = np.arange(2026, 2046)
future_months = np.tile(np.arange(1, 13), len(future_years))
future_years_full = np.repeat(future_years, 12)
future_preds = []

last_input = dataset_scaled[-look_back:].reshape(look_back, 1)

for year, month in zip(future_years_full, future_months):
    season = get_season(month)
    sin_m, cos_m = month_sin_cos(month)

    input_seq = last_input.flatten()
    input_full = np.concatenate((input_seq, [season, sin_m, cos_m])).reshape(1, -1)

    pred_scaled = model.predict(input_full)
    pred_temp = scaler.inverse_transform(pred_scaled.reshape(-1, 1))[0][0]

    future_preds.append([year, month, pred_temp])

    norm_pred = scaler.transform([[pred_temp]])[0][0]
    last_input = np.roll(last_input, -1)
    last_input[-1, 0] = norm_pred

with open("linear_predikcije_2026_2045.txt", "w") as f:
    f.write("Godina, Mjesec, Predikcija temperature\n")
    for pred in future_preds:
        f.write(f"{pred[0]}, {pred[1]}, {pred[2]:.2f}\n")

print("Predikcije za 2026-2045 sačuvane u 'linear_predikcije_2026_2045.txt'.")
